In [2]:
pwd

u'E:\\Github\\ipython.notebook.dir'

In [36]:
%pylab
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.qinv = tf.quaternion_inverse
tf.qmul = tf.quaternion_multiply

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
'测试： 左乘表示固定轴旋转，√'
axes='sxyz'
x,y,z=10,20,34
ms=tf.e2m(x,y,z, axes)
print ms

axes='rxyz'
mr=tf.e2m(x,y,z, axes)
print mr

'单轴旋转 r/s 应该是同样结果'
axes='rxyz'
mx=tf.e2m(x, 0, 0, axes) 
my=tf.e2m(0, y, 0, axes)
mz=tf.e2m(0, 0, z, axes)
print '---------------'
# print mx
# print my
# print mz
mleft=mz.dot(my).dot(mx)
mright=mx.dot(my).dot(mz)
'助记：rot-frame 对应 right-multiply'
assert np.allclose(ms, mleft)
assert np.allclose(mr, mright)

[[-0.34628631  0.8653904   0.36219505  0.        ]
 [ 0.21590915  0.44923616 -0.86693143  0.        ]
 [-0.91294525 -0.22200526 -0.34241004  0.        ]
 [ 0.          0.          0.          1.        ]]
[[-0.34628631 -0.21590915  0.91294525  0.        ]
 [-0.02248604  0.97478615  0.22200526  0.        ]
 [-0.93785936  0.05634886 -0.34241004  0.        ]
 [ 0.          0.          0.          1.        ]]
---------------


In [16]:
'some utils, 从下面 cell 剪切过来 //2016-5-13 10:16:00'

from lxml import etree
psr = etree.XMLParser(remove_blank_text=True)

def plotRvecs(fname):
    if fname.endswith('.csv'):
        pose_mat = loadtxt(fname, delimiter=',')
        rvecsDepth = pose_mat[:, 3:]
        return plot(rvecsDepth)
#     elif fname.endswith('.xml'):
#         tree = etree.parse(fname, parser=psr)
#         rt = tree.getroot()
#         tvecsColor = []
#         for i, iframe in enumerate(rt):
#             tvecsColor.append([float(num) for num in iframe.xpath('t/data')[0].text.split()])
#         tvecsColor = np.asarray(tvecsColor)
    pass

def plotTvecs(fname):
    if fname.endswith('.csv'):
        pose_mat = loadtxt(fname, delimiter=',')
        tvecsDepth = pose_mat[:, :3]
        return plot(tvecsDepth, '')
#     elif fname.endswith('.xml'):
#         tree = etree.parse(fname, parser=psr)
#         rt = tree.getroot()
#         tvecsColor = []
#         for i, iframe in enumerate(rt):
#             tvecsColor.append([float(num) for num in iframe.xpath('t/data')[0].text.split()])
#         tvecsColor = np.asarray(tvecsColor)/1000
#         plot(tvecsColor)
    pass

# def diffCsvs(fn1, fn2, minLen = inf):
# def diffCsvs(fn1, fn2, minLen = slice(None)):
# def diffCsvs(fn1, fn2, rng = slice(None), minLen = inf): #主推 rng; 保留 minLen, 向后兼容
def diffCsvs(fn1, fn2, rng = slice(None), minLen = inf, assoFn = ''): #增加 assoFn, 是 TUM associate.py 输出的关联描述txt, 尽量向后兼容
    extStr = '.csv'
#     assert fn1.endswith(extStr) and fn2.endswith(extStr) #没必要, csv, txt 都行 @2018-5-3 17:18:33
    '2 pose_mats:'
    if assoFn == '':
        '若不考虑同步问题'
        m1 = loadtxt(fn1, delimiter=',')[:, :7]
        m2 = loadtxt(fn2, delimiter=',')[:, :7]
    else: #if assoFn != '': #若真指定了 zc-associate-res.txt
        '''
        ---------------考虑同步问题: 启用TUM 关联描述文件, 2016-12-11 20:52:42
        近期主要应用场景: rgb-cb-gt.csv 与 depth 不同步, 可能夸大误差评估结果, e.g.: cb5x8-cu399c-newIntr-icp0\zcpose-1002.csv
        '''

        '强制 fn1=rgb...'
        assert 'rgb-cb-gt.csv'in fn1, 'if assoFn != "", fn1 must be .../rgb-cb-gt.csv'
        m1 = loadtxt(fn1, delimiter=',')[:, :7] #本来 getExtrs.ipynb 生成的 rgb...csv 是8列, 但是目前感觉第8列 ts 没用 #2016-12-12 01:21:38
        m2 = loadtxt(fn2, delimiter=',')[:, :7]
        print '--m1.shape, m2.shape: ', m1.shape, m2.shape
        
        assert os.path.exists(assoFn), 'assoFn NOT exists~'
        assoMat = loadtxt(assoFn,  delimiter=' ', dtype=str) #加载 zc-associate-res.txt

        '反推 TUM-proj 目录'
        projDir, _ = os.path.split(assoFn)
        '+++++++++++++++rgb 描述txt, 【注意: 我这里用 rgb 图像, 棋盘格结果做 GT, 所以 assoFn 文件作用不仅是 rgbd-slam 同步输入那样'
        rgbTxtfn = os.path.join(projDir, 'rgb.txt')
        assert os.path.exists(rgbTxtfn), 'rgbTxt NOT exists~'
        rgbTxtmat = loadtxt(rgbTxtfn, delimiter=' ', dtype=str) #加载 rgb.txt
        rgbtxtTsvec = map(float, rgbTxtmat[:,0]) #第一列时间戳
        
        'depth 描述txt'
        depthTxtfn = os.path.join(projDir, 'depth.txt')
        assert os.path.exists(depthTxtfn), 'depthTxt NOT exists~'
        depthTxtmat = loadtxt(depthTxtfn, delimiter=' ', dtype=str) #加载 depth.txt
        deptxtTsvec = map(float, depthTxtmat[:,0]) #第一列时间戳

        '同步后的, 深度序列 ts(也即文件名)'
        deptsvec = map(float, assoMat[:,0]) #depth-ts-vec in assoFn
        rgbtsvec = map(float, assoMat[:,2])
        print '--len(deptsvec), len(rgbtsvec): ', len(deptsvec), len(rgbtsvec)
        seleDepInds = [deptxtTsvec.index(dts) for dts in deptsvec] #selected depth indices
        seleRgbInds = [rgbtxtTsvec.index(rgbts) for rgbts in rgbtsvec]
        assert len(seleRgbInds) == len(seleDepInds), '同步后还不一样长？'
        
        '同步后的【相机姿态序列】'
        m1 = m1[seleRgbInds] #记得: fn1, m1 对应 rgb
#         print 'm1[918]', m1[918]
        m2 = m2[seleDepInds]
        print '--AFTER sync, m1.shape, m2.shape: ', m1.shape, m2.shape
        
        '去除 全零行'
        non0flags = np.array([not (allclose(row1[:7], [0]*7) or allclose(row2[:7], [0]*7)) for row1, row2 in zip(m1, m2)])
        print '--non0flags: non0vs.0s, cnt: ', sum(non0flags), len(m1)-sum(non0flags)
#         print non0flags[918:922] 
        
        m1 = m1[non0flags]
        m2 = m2[non0flags]
        print '--AFTER non0flags, m1.shape, m2.shape: ', m1.shape, m2.shape
        
        '''
        m1 现在是 cboard/world/global)->cam (g2c), 还要改成 c->g (c2g)
        若 Vci->Vc0: Vc0=R0*Ri.T(Vci-ti)+t0
        '''
        m1tvecs = m1[:, :3]
        m1rvecs = m1[:, 3:]
        m1rmats = np.array([tf.q2m([q[1], q[2], q[3], q[0]]) for q in m1rvecs]) #N*(4x4)
        
        m1t0 = m1tvecs[0]
        m1r0 = m1rmats[0]
        
        m1rmats = np.array([m1r0.dot(ri.T) for ri in m1rmats]) #R0*Ri.T
        m1tvecs = np.array([m1t0 - ri[:3, :3].dot(ti) for ri, ti in zip(m1rmats, m1tvecs)]) #这里的 m1rmats 已更新过, 是 R0*Ri.T 了
        
        m1rvecs = np.array([tf.m2q(ri) for ri in m1rmats])
        print '--m1tvecs.shape, m1rvecs.shape: ', m1tvecs.shape, m1rvecs.shape
        
        #m1 = np.hstack((m1tvecs, m1rvecs[:, 3], m1rvecs[:, :3])) #再整合回 N*7 mat
        print '--m1tvecs.shape, m1rvecs[:, (3,0,1,2)].shape: ', m1tvecs.shape, m1rvecs[:, (3,0,1,2)].shape
        m1 = np.hstack((m1tvecs, m1rvecs[:, (3,0,1,2)])) #上面错, [:, 3] 得到的是 1D-array
        pass
    
    '自动截断： 取两列表中短的为准'
    minLen = min(len(m1), len(m2), minLen) #若用户指定更小的 minLen, 则用它, 默认值 inf
    print '--diffCsvs(), len(m1), len(m2), minLen=', len(m1), len(m2), minLen
    
    m1 = m1[:minLen] #前面取 min 操作, 使 minLen 不可能为 inf
    m2 = m2[:minLen]
    m1 = m1[rng]
    m2 = m2[rng]
    
    print '--mean-abs-diff(各列):', np.abs(m1-m2).mean(axis=0)
    'tvecs:'
    m1tvecs = m1[:, :3]
    m2tvecs = m2[:, :3]

    'R: wxyz 序'
    m1rvecs = m1[:, 3:]
    m2rvecs = m2[:, 3:]
    m1rmats = np.array([tf.q2m([q[1], q[2], q[3], q[0]]) for q in m1rvecs])
    m2rmats = np.array([tf.q2m([q[1], q[2], q[3], q[0]]) for q in m2rvecs])
    
    '两序列起始帧: m1rmats[0], m2rmats[0] 未必一致, 所以要先处理成二者处于同一参考坐标系, 修改m2, 与m1对齐. 2016-5-26 22:35:05'
    m1t0 = m1tvecs[0]
    m1r0 = m1rmats[0]
    m2t0 = m2tvecs[0]
    m2r0 = m2rmats[0]
    
    '纸上推导: 2Ri = 1R0 * 2R0.T * 2Ri; 2ti = 1R0 * 2R0.T * (2ti-2t0) + 1t0; 2016-5-26 22:53:04'
    m2rmats = np.array([m1r0.dot(m2r0.T).dot(ri2) for ri2 in m2rmats])
    m2tvecs = np.array([m1r0.dot(m2r0.T)[:3,:3].dot(ti2-m2t0) + m1t0 for ti2 in m2tvecs])
    
    deltaQs = [tf.m2q(m1m.dot(m2m.T)) for m1m, m2m in zip(m1rmats, m2rmats)] #暂时没用上
    
    '绘制m1,m2 txyz'
    figure(figsize=(10,10))
    subplot(311)
    title('m1&m2-tvecs')
    m1ts = plot(m1tvecs)
    m2ts = plot(m2tvecs)
    legend(m1ts+m2ts, ['m1x', 'm1y', 'm1z', 'm2x', 'm2y', 'm2z'])

    '求解 ATE, 参照 evaluate_ate.py, 确信算法规则'
    #ate = norm(m1tvecs - m2tvecs) / sqrt(len(m1tvecs)) #可以, 但是下面改成也画每一帧误差, 所以不统着算
    ate = norm(m1tvecs - m2tvecs, axis=1) #每一帧二者的 tvec 偏差
    rmse = norm(ate) / sqrt(len(m1tvecs))
    print 'ATE-RMSE:', rmse
    ax_ate = subplot(312)
#     figure() #单独开一个 fig //CuFusion2 论文用
#     ticklabel_format(style='sci', scilimits=(0,4)) #科学计数法
    title('m1&m2-ATE')
#     plot(tnorms)
    font_sz = 32
    xlabel('frame number', fontsize = font_sz)
    ylabel('Absolute Trajectory Error (ATE) (mm)', fontsize = font_sz)
    #tick_params(labelsize = 'large') #缩小了看 仍然太小，看不清
    #tick_params(labelsize = font_sz) #数轴上数字的字体大小
    plot(ate, '-k.') #带点折线, 非虚线
#     for i in xrange(0, len(tnorms), 5): #每隔几帧, 输出误差值, 调试观察
#         annotate('%.5f'%tnorms[i], (i, tnorms[i]))
#     xlim(-1, 60)
#     ylim(0, 2.7)
    
    legend(['ATE=%.4fm'%rmse], loc='upper right')
    return m1tvecs, m1rmats, m2tvecs, m2rmats, ate
    pass

In [131]:
'''
测试: TUM 同步描述文件的一些处理
【此cell 放弃？ 代码已经融入上面 diffCsvs？ 记不清了 2016-12-27 17:21:00】
'''
assoFn = r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\zc-associate-res.txt'
'反推 proj 目录'
projDir, _ = os.path.split(assoFn)
'depth 描述txt'
depthTxtfn = os.path.join(projDir, 'depth.txt')
assert os.path.exists(depthTxtfn), 'depthTxt NOT exists~'
depthTxtmat = loadtxt(depthTxtfn, delimiter=' ', dtype=str) #加载 depth.txt
depthTsvec = map(float, depthTxtmat[:,0])
print 'len(depthTsvec): ', len(depthTsvec)

assoMat = loadtxt(assoFn,  delimiter=' ', dtype=str) #加载 zc-associate-res.txt

'同步后的, 深度序列 ts(也即文件名)'
deptsvec = map(float, assoMat[:,0]) #depth-ts-vec in assoFn
rgbtsvec = map(float, assoMat[:,2])
print 'len(deptsvec), len(rgbtsvec): ', len(deptsvec), len(rgbtsvec)
selectedInds = [depthTsvec.index(dts) for dts in deptsvec]
print 'len(selectedInds): ', len(selectedInds)

m2fn = r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\zcpose.kf.csv'
m2mat = loadtxt(m2fn, delimiter=',')
m2sync = m2mat[selectedInds]
m2sync.shape #√

len(depthTsvec):  1736
len(deptsvec), len(rgbtsvec):  1617 1617
len(selectedInds):  1617


(1617, 7)

In [51]:
'''
2016-5-13 10:32:18
---------------测试对比kinfu.orig(记为 m1) & 棋盘格外定标 (直接用, 不是做初值, -icp 0)(记为 m2)
'''
'+++++++++++++++《立方体重建结果中“棱边钝化”误差分析.docx》'
m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-t0(-0.3)\zcpose.csv' #kinfu.orig, 原本就 f1002, 即M1
# m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-pcc\zcpose-1002.csv' #kinfu.pcc, 截断 1002, 即M3
m2fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-icp0\zcpose-1002.csv' #kinfu+cb, 截断 1002, 即M21

'+++++++++++++++理论应二者全等, 实际也是, 2016-5-24 21:51:28'
m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-pcc\zcpose.csv' #kinfu.pcc, 不截断
m2fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-pcc-icp0-ero5\zcpose.csv' #pcc 先得到外参, 又重新 -icp0 跑, 

'+++++++++++++++regObj=2, |d-T*s| 与 |T*d-s| (后缀 -g2c) 对比, 2016-5-26 21:58:48'
'好数据上, 正确性测试'
m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399-tz-kf-reg=0\zcpose.csv' #kf.orig
m2fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399-tz-kf-reg=2\zcpose.csv' #regObj=2
m2fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399-tz-kf-reg=2-g2c\zcpose.csv' #-g2c
minLen = 150 #这个 -tz 数据后来我晃动了, 前期基本平移(<200)的时候, M0-M2误差在 7mm, 非常小

'+++++++++++++++合成数据做 groundtruth, 用于测试 kf etc. 算法的 (R,t) 精度'
'①, synDepth-tz3, reset'
'②'
m1fn = r'E:\Github\pcl\doc\tutorials\content\sources\pcl_visualizer\zcSyntheticCube\synDepth-tx3\synDepthPose.csv'
m2fn = r'E:\Github\pcl\_build.vc10\bin\synDepth-tx3-kf\zcpose.csv' #kf, 0.00031m
'③'
m1fn = r'E:\Github\pcl\doc\tutorials\content\sources\pcl_visualizer\zcSyntheticCube\synDepth-tx3-2f\synDepthPose.csv'
m2fn = r'E:\Github\pcl\_build.vc10\bin\synDepth-tx3-2f-kf\zcpose.csv' #kf, 0.00718m
'④'
m1fn = r'E:\Github\pcl\doc\tutorials\content\sources\pcl_visualizer\zcSyntheticCube\synDepth-rz3-2f\synDepthPose.csv'
m2fn = r'E:\Github\pcl\_build.vc10\bin\synDepth-rz3-2f-kf\zcpose.csv' #kf, 0.01241m
'⑤'
m1fn = r'E:\Github\pcl\doc\tutorials\content\sources\pcl_visualizer\zcSyntheticCube\synDepth-rx3-2f\synDepthPose.csv'
m2fn = r'E:\Github\pcl\_build.vc10\bin\synDepth-rx3-2f-kf\zcpose.csv' #kf, 0.08362m
'⑥'
m1fn = r'E:\Github\pcl\doc\tutorials\content\sources\pcl_visualizer\zcSyntheticCube\synDepth-ry3\synDepthPose.csv'
m2fn = r'E:\Github\pcl\_build.vc10\bin\synDepth-ry3-kf\zcpose.csv' #kf, 0.00125m
'⑦, 9张图'
m1fn = r'E:\Github\pcl\doc\tutorials\content\sources\pcl_visualizer\zcSyntheticCube\synDepth-rt-3f\synDepthPose.csv' #groundtruth
m2fn = r'E:\Github\pcl\_build.vc10\bin\synDepth-rt-3f-kf\zcpose.csv' #kf, 0.000426m

'+++++++++++++++kf.orig 对比 IMUD-fusion //2016-7-14 10:39:48'
m1fn = r'E:\Github\pcl\_build.vc10\bin\monitor-0708-0-kf-png\zcpose.csv'
m2fn = r'E:\Github\pcl\_build.vc10\bin\monitor-0708-0-kf-imud\zcpose.csv'

m1fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0714-1-kf-png\zcpose.csv"
m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0714-1-kf-imud\zcpose.csv"

# m1fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf\zcpose.csv"
# # # m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf-imud-iter3\zcpose.csv" #imud 仅3次迭代
# # # m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf-imud-iter5\zcpose.csv"
# m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf-imud\zcpose.csv"
# # m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf-iter5\zcpose.csv"
# minLen = inf #700

# m1fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-d688-781-kf\zcpose.csv"
# m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-d688-781-kf-imud\zcpose.csv"
# # m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-d688-781-kf-imud-tcurr\zcpose.csv" #用 tcurr 不用 tcurr_back, 但是结果更差

'+++++++++++++++kf.pcc 对比 IMUD //2016-7-15 16:28:07'
m2fn = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-pcc\zcpose.csv'
# m2fn = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-pcc-imud\zcpose.csv' #imu 用理想旋转矩阵变换到 cam 坐标系
# m2fn = r'E:\Github\pcl\_build.vc10\bin\monitor-0714-1-pcc-imud-axxb\zcpose.csv' #imu 用axxb 标定矩阵
minLen = inf

# m1fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-kf\zcpose.csv"
# m2fn = r"E:\Github\pcl\_build.vc10\bin\monitor-0715-0-pcc\zcpose.csv"

# '一般数据上, 精度测试, 数据1'
# m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-t0(-0.3)\zcpose.csv' #kf.orig
# m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-reg=2\zcpose.csv' #regObj=2
# minLen = 70
# m2fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-reg=2-g2c\zcpose.csv' #-g2c
# # m2fn = r'E:\Github\pcl\_build.vc10\bin\zcpose.csv'
# minLen = 32

# '精度测试, 数据2, 其实是截取数据1 f600~700。 //2016-5-27 17:11:51'
# m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-f600~700-reg=0\zcpose.csv' #kf.orig
# m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-f600~700-reg=2\zcpose.csv'  #regObj=2
# m2fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-f600~700-reg=2-g2c\zcpose.csv' #-g2c
# # m2fn = r'E:\Github\pcl\_build.vc10\bin\zcpose.csv'
# minLen = inf

'+++++++++++++++kf 对比 ahc(peac)立方体定位 //2016-9-29 14:46:51'
# m1fn = r'E:\Github\pcl\_build.vc10\bin\00f80c399-kf-f0reg=0-inp_src_bad\zcpose.csv'
m1fn=r'E:\Github\pcl\_build.vc10\bin\00f80c399-kf-f0reg=0\zcpose.csv' #早先某 kf.orig 结果, //似乎是默认内参
m1fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\zcpose-kf.csv' #kf.orig: 新内参, kinfu 命令行: -dbg1 --depth-intrinsics 579.267,585.016,311.056,242.254 -save_pose zcpose.csv -sv -png-dir "D:/Users/zhangxaochen/Documents/geo-cube399/00f80c-raw_frames-1-45-3p"
m1fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\zcpose-pcc.csv' #kf.pcc: 命令行：-dbg1 -icp_impl "sgf_cpu" -cc_norm_prev 02 --depth-intrinsics 579.267,585.016,311.056,242.254 -save_pose zcpose.csv -sv -png-dir "D:/Users/zhangxaochen/Documents/geo-cube399/00f80c-raw_frames-1-45-3p"
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-vga.csv' #施密特正交化, #ate=9mm, 29帧误差突变
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-vga-opt-ax_norm.csv' #用优化方法(svd)做正交化, 三面法向初始轴, #ate=4mm,误差均匀增大,是f2f后果
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-vga-opt-ax_intersline.csv' #三面交线做初始轴, 居然与上面【全等】
# # '不正交化, 当做 procrustes 问题 #与上面ax_norm【几乎全等】, ate=0.03mm, 说明单顶角做到了极限'
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-vga-procrustes.csv'
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-vga-procrustes-mult_vert.csv' #【多顶角】优化, ate=3.5mm
# # m1fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-qvga.csv' #对比【施密特vga】：ate=0.17mm
m1fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-qvga-opt-ax_norm.csv' #与vga相比: ate=0.57mm, 可认为精度几乎一样
m1fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-qvga-procrustes-mult_vert.csv' #ate=0.49
minLen = inf

'''
+++++++++++++++glGenSynData仿真数据 vs. peac //2016-10-21 11:18:05
这次仍主要（仅）关注三邻面可见视角下的结果
'''
'仅绕Z轴, 正圆'
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep0\syn_pose.csv' #此轨迹没法用; 弃用
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep1\syn_pose.csv' #GTruth
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep1\pose-vga.csv' #ate=0.26mm, 可认为近似==0
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep1\pose-qvga.csv' #ate=0.34mm, 仍可认为==0
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep1-pcc\zcpose.csv' #总体ate=6mm, 前27帧ate=2mm; 居然误差比 peac 大...是因为f2m时model噪声导致配准误差?
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep1-kf\zcpose.csv' #ate=0.2m, 150帧开始巨大漂移
minLen=inf#27

'绕X/Z轴, 起伏环绕, 非正圆'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep2\syn_pose.csv' #GTruth
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep2\pose-qvga.csv' #ate=0.15mm, 近似0
'下面2组只看前~80帧误差'
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep2-kf\zcpose.csv' #ate=2.39mm
# m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep2-pcc\zcpose.csv' #ate=2.69mm, 比 kf高, 说明此数据上边缘权重拖后腿?
'下面看整段误差, kf.orig ~130帧漂移太大, 不管; pcc约束住了; 2016-12-2 16:35:48'
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep2-pcc-vsz1.5-td15\zcpose.csv' #ate=5.16mm, vsz1.5怎么反而变差了？

'故意乱搞内参：609.275,609.275 -->555,555，观察对轨迹的影响'
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep2\pose-qvga-fx555.csv' #ate=8mm, 内参fxy=555, 确实影响轨迹, 因为影响了3D空间尺度
# m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep2-kf-fx555\zcpose.csv' #ate=9.5mm
# m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep2-pcc-fx555\zcpose.csv' #ate=9.6mm
minLen=29

'添加 Bunny, 非正圆,'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep3\syn_pose.csv' #GTruth
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep3\pose-vga.csv' #ate=2.1mm, fid=2时, 丢了一个顶角, 原因: 't_{num}=3000' 默认太大了
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep3\pose-qvga.csv' #ate=2.1mm
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep3\pose-vga-tnum2000.csv' #ate=0.95mm
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep3-pcc\zcpose.csv' #ate=2.9mm, 若仅前40帧, ate=1.35mm
# m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep3-pcc-fx555\zcpose.csv' #ate=108.9mm
minLen=40

'添加中国龙 dragon, 手控, 旋转step=0.5°很小'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep4\syn_pose.csv'
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep4\pose-vga.csv' #ate=0.184mm, t_{num}=3000
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep4\pose-vga-tnum2000.csv' #ate=0.194mm; 这次降低 tnum, 误差居然稍微增大了
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep4-pcc\zcpose.csv' #ate=4.83mm; 若 f0~153, ate=2.94mm
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep4-kf\zcpose.csv' #ate=3.04mm, 这里 kf 居然优于 pcc!! 若 f0~153, ate=1.76mm 
minLen=153#inf

'添加犰狳, '
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep5\syn_pose.csv'
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep5\pose-vga.csv' #0.27mm, @f130
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-kf\zcpose.csv' #ate=5.08mm, 仍然 kf 优于 pcc!!! @f130:2.09mm
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-pcc\zcpose.csv' #ate=6.52mm, @f130:3.19mm
minLen=130#inf

'用@犰狳数据, 这次故意搞乱内参(585), 观察 ours/pcc/kf 哪个更差'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep5\syn_pose.csv'
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep5\pose-vga-f585.csv' #3.91mm
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-kf-f585\zcpose.csv' #f130:4.20mm
'内参555, 605, etc.'
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep5\pose-vga-f555.csv' #f17 断掉, 因不满足5°正交阈值判定
m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\synDep5\posepose-vga-f605.csv' #0.74mm, 虽然内参差了一点, 但是误差可接受 #之前0.27mm
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-kf-f605\zcpose.csv'#2.46mm, 误差相应增大一点, 之前 2.09mm
'volume=1.5m, 故意小体素, 看精度变化'
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-kf-vsz1.5\zcpose.csv' #4.95mm, 轨迹比 vsz=3m时略有提升, 无漂移问题
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-pcc-vsz1.5\zcpose.csv' #6.123mm, f130:2.89mm
m1fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-GT-vsz1.5\zcpose.csv' #ate=0mm, 说明确实用了GT, 但是mesh 仍然很差
m2fn=r'E:\Github\pcl\_build.vc10\bin\synDep5-GT-kf-vsz1.5\zcpose.csv'
minLen=130#inf

'''
2016-12-12 04:49:07
+++++++++++++++重写【同步】策略之后, 认为 depth & rgb 不存在非同步问题, 于是看精度
但是, 目前看到 rgb-cb-gt 反而不准, 不光滑 //2016-12-12 04:49:40
'''
assoFn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\zc-associate-res.txt'
# assoFn=''
m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\rgb-cb-gt.csv'
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\zcpose.kf.csv' #vs.cb-gt:42.3mm; 
# m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\zcpose.pcc.csv' #vs.cb-gt:28.9mm
rng = slice(113, 153)
# m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts\pose-ahc.csv' #vs.cb-gt:13.0mm
rng = slice(40)

'注意这组是 -ts2: 抽取上面数据的 ..761465~..263081 深度图, rgb取广一点涵盖此时间段即可, '
assoFn=''
assoFn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\zc-associate-res.txt'
m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\rgb-cb-gt.csv'
# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\zcpose.kf.csv' #cb-gt:13.3mm
# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\zcpose.pcc.csv' #cb-gt:12.7mm; kf:1mm; 
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc-single_vert.csv' #cb-gt:12.9mm; pcc:4.9mm; kf:5.1mm
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc-multi-vert.csv' 
            #cb-gt:12.68mm; pcc:5.1mm; kf:5.26mm; -single_vert:0.5mm
rng = slice(None)

'测试 zcRefinePlsegParam 与 extractedPlanes 在计算姿态上的差异, //2016-12-27 20:12:49'
assoFn=''
m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc-extractedPlanes.csv'
m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc-zcRefinePlsegParam.csv'
    #↑-此csv等价于 pose-ahc-multi-vert.csv #比-extractedPlanes: ate=0.89mm
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc-zcRefinePlsegParam-zcFindOrtho3tup.csv'
    #比仅 -zcRefinePlsegParam: ate=0.02mm, 但文本看起来差异貌似没这么小啊？
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose.csv'

'kinfu中加入了立方体定位方法'
'@05f80c'
# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-pcc.csv'
m1fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-pcc-trunc0.csv' #3mm, 轨迹差别小, 但模型改善显著
# m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-kf-ahc-nog2o.csv' #1271帧, 应该是1157啊...可能当时有bug
# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-kf-ahc-nog2o-trunc0.csv'
    #与无 -trunc0: 1.9mm, 与pcc-trunc0: 12.4mm, 差别还是不小, 每次检测到立方体顶角就突变, 注意看帧序号
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-kf-ahc-graph-opt-trunc30-485.csv' #13.8mm
    #http://imgur.com/a/7kdwE
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\zcpose-kf-ahc-graph-opt-trunc0.csv'
    #pcc-truncc0: 13.97mm; ahc-graph-opt-trunc30-485: 1.46mm; ahc-nog2o-trunc0: 3mm
'↓-对应重建结果最优！！！见《三维重建结果与vivid 3D 扫描仪groundtruth真值对比测试》-【m6】相关'
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\output-cuicp\zcpose.csv'
    #pcc-truncc0: 14.52mm; ahc-nog2o-trunc0: 4mm
    
'@10f80 没 crop 干净'
# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\10f80c-raw_frames-ts\zcpose-kf.csv'
# m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\10f80c-raw_frames-ts\zcpose-ahc.csv' #10.8mm
# m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\10f80c-raw_frames-ts\zcpose-ahc-graph-opt.csv' #12.3mm
# m2fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\10f80c-raw_frames-ts\zcpose-kf-t30.csv' #kf:6.3mm, 说明 -t0/t30 对轨迹影响很小
# rng = slice(1434)

# m1fn=r'E:\Github\pcl\_build.vc10\bin\zcpose.csv'
# m2fn=r'E:\Github\pcl\_build.vc10\bin\zcpose-ahc.csv'
# m2fn=r'E:\Github\pcl\_build.vc10\bin\zcpose-ahc-graph-opt.csv'

'+++++++++++++++cuOdo & bdr 对比评估 @2017-4-21 15:33:31'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\syn_pose.csv'
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\cuOdo&bdr\p7-kfLS-t25-vs1.5.csv' #4.79mm; 居然最小？
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\cuOdo&bdr\p7-bdr-t25-vs1.5.csv' #GT: 7.41mm; kfLS: 3.97mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\cuOdo&bdr\p7-cu-t25-vs1.5-tv11.csv' #与 GT: 1.63mm; bdr: 7.50mm; kfLS: 4.68mm
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\cuOdo&bdr\p7-cu-t25-vs1.5-tv11-bdrNm.csv' #GT: 1.56mm; 跟上面比: 0.185mm
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep6\cuOdo&bdr\vs1-res384\p7-bdr-t5.4.csv' #GT: 1.99mm
# rng = slice(99)

'''
@2017-7-9 23:59:58
+++++++++++++++重新以 cu4armadillo 生成仿真数据, 
再加载之后 -csv_rt_hint -synRtFn syntheticRT.txt --icp 0 参数, 却发现严重漂移 
答：生成数据时 syn_pose.csv 受 gl-push/pop-matrix 误操作影响, 导致轨迹 GT 是错的; 现已修复
'''
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\syn_pose.csv'
m2fn=r'E:\Github\pcl\_build.vc10\bin\zcpose.csv'

'''
@2017-7-17 23:11:03
+++++++++++++++跑仿真数据 synDep8~10 时, 误用某内参, 对比其跟 GT 内参对轨迹的影响
'''
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-bdr-t25-vs1.csv'
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr-myKinectV1\kinfu_poses-bdr-t25-vs1.csv' #26.4mm, 有点大

'''---------------synDep8~10, kf/bdr/ours 分别与 GT 对比
发现 -e2c5cm 在仿真数据上有毛病, 可能太稀疏了 @2017-7-18 16:39:04
'''
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\syn_pose.csv'
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-kf-t25-vs1.csv' #3.17
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-t25-vs1.csv' #6.37mm
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\liv.klg-conf0.csv' #ElasticFusion 7.08
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.csv' #5.46
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-12.csv' #1.46
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c2cm-remake.csv' #1.58
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c2cm-amp4-remake.csv' #1.45
# m2fn=r'E:\geo-cu399-pen\data-archive\01-armadillo\nicp_poses.csv'
'↓--@2018-10-25 23:38:53'
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-v21.5.8.csv' #cu2 #1.668mm
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-v21.5.8-2-e2c=1cm.csv' #cu2 #1.563mm @e2c=1cm 其实差异 0.1mm 可忽略
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu2-tv21.5.9-vs06-t05.csv' #cu2 #1.666mm
'↓--完全按论文 cu2 草稿中的公式 @2018-11-30 01:38:27'
m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu2-tv21.6.0-vs06-t05.csv' #1.642 

'↓--在新外参 -init_rt 下, 重新生成 kf, bdr 的结果 (且不同 tdist) @2018-11-27 11:06:42 '

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-kf-vs06-t10=BAD.csv' #131.6mm, 主要在 f43 突然漂移, 后面稳定, 最终mesh还行
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-kf-vs06-t05=BAD.csv' #1005mm, 非常差, f34 飘走, 无法恢复, 说明 td 小了太差
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-kf-vs06-t20.csv' #3.57mm, 就很好 (虽然mesh很模糊)
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-kf-vs06-t25.csv' #3.72mm, 没之前 kf-t25-vs1 好, (diff<1mm, 其实没差别)

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-vs06-t10=WRONG.csv' #131.6mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-vs06-t05=WRONG.csv' #1004.5mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-vs06-t05.csv' #5.635mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-vs06-t10.csv' #5.464mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-bdr-vs06-t25.csv' #5.140mm, arma数据上, 更大的 td反而更好

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-v11-t05-e2c=2cm.csv' #1.658mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-v11-t10-e2c=2cm.csv' #1.717mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-cu-v11-t25-e2c=2cm.csv' #1.861mm
    
# m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-s2s[0.01,0.5,1]-t25.csv' #17.155mm
# # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\cu&bdr\kinfu_poses-s2s[0.01,0.5,1]-t05.csv' #5.702mm
# # rng = slice(199)

'''---------------synDep9, dragon 中国龙'''
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\syn_pose.csv'
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-kf-t25-vs1.csv' #4.20
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-bdr-t25-vs1.csv' #6.70mm
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\liv.klg-conf0.csv' #ElasticFusion 7.97
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.csv' #16.1mm
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-12.csv' #1.82
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c2cm-amp4-remake.csv' #1.67
# # m2fn=r'E:\geo-cu399-pen\data-archive\02-dragon\nicp_poses.csv'
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-v21.5.9.csv'
# # m2fn=r"E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-s2s[0.01,0.5,1]-t05.csv" #4.381mm

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-kf-vs06-t05.csv' #2.96mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-kf-vs06-t10.csv' #2.51mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-kf-vs06-t25.csv' #3.39mm

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-bdr-vs06-t05.csv' #2.916mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-bdr-vs06-t10.csv' #2.937mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\cu&bdr\kinfu_poses-bdr-vs06-t25.csv' #4.681mm, 

'''---------------synDep10, bunny 兔子'''
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\syn_pose.csv' 
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-kf-t25-vs1.csv' #3.91
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-bdr-t25-vs1.csv' #5.12mm
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\liv.klg-conf0.csv' #ElasticFusion 6.55
# # # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.csv' #2.39
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-12.csv' #1.47
# # # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c2cm-amp4-remake.csv' #1.25
# # m2fn=r'E:\geo-cu399-pen\data-archive\03-bunny\nicp_poses.csv'
# # m2fn=r"E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-s2s[0.01,0.5,1]-t05.csv" #6.644mm, s2s 的 f2m
# # m2fn=r"E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-s2s[0.01,0.5,0]-t05.csv" #118.665mm, f2f 在仿真上太差了, 但是 f2m 在真实噪声数据上又太差
# # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# # m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-v21.5.8.csv' #1.728mm

# # # m1fn=r'E:\geo-cu399-pen\turntable\flycat1c-raw_frames-ts\cu&bdr\kinfu_poses-bdr-t5-vs1.csv'
# # # m2fn=r'E:\geo-cu399-pen\turntable\flycat1c-raw_frames-ts\cu&bdr\kinfu_poses-bdr-t10-vs1.csv' #5mm

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-kf-vs06-t05.csv' #4.17mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-kf-vs06-t10.csv' #3.38mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-kf-vs06-t25.csv' #3.35mm

# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-bdr-vs06-t05.csv' #4.320mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-bdr-vs06-t10.csv' #3.767mm
# m2fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\cu&bdr\kinfu_poses-bdr-vs06-t25.csv' #3.444mm

# # '''
# # @2017-8-14 13:41:33
# # -plfilt 是去除长方体基座, 仅用上层 obj 做定位+重建; 定位本应用全图, 偷懒暂用 filt 后的 obj 部分
# # ate=5.37mm, 还比较准
# # '''
# # # m1fn=r'E:\geo-cu399-pen\turntable\bunny1c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.csv'
# # # # m1fn=r'E:\geo-cu399-pen\turntable\bunny1c-raw_frames-ts\cu&bdr\kinfu_poses-kf-t25-vs1.csv' #14mm
# # # m2fn=r'E:\geo-cu399-pen\turntable\bunny1c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-plfilt.csv'

# # # m1fn=r'E:\geo-cu399-pen\turntable\buddha1c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake.csv' #3.26mm
# # # m2fn=r'E:\geo-cu399-pen\turntable\budda1c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t25-tv11-vs1-e2c5cm-remake-plfilt.csv'
# # m2fn=r'E:\geo-cu399-pen\turntable\buddha1c-raw_frames-ts\cu&bdr\kinfu_poses-v21.5.8.csv'

# # '''
# # @2017-11-15 19:42:44
# # cufusion 继续改进, 尝试 g2o 图优化
# # '''
# # # m1fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# # # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\zcpose-ahc-g2o.csv'
# # # m2fn=r'E:\geo-cu399-pen\turntable\tooth1c-raw_frames-ts\cu&bdr\kinfu_poses-kf-t25-vs1.csv'

# m1fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\g2o\kinfu_poses.csv'
# m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\g2o\zcpose-ahc-g2o.csv'
# # # m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-cu-t25-tv11-vs1.5.csv'

# # m1fn=r'E:\geo-cu399-pen\turntable\antenna1c-raw_frames-ts\g2o\kinfu_poses.csv'
# # m2fn=r'E:\geo-cu399-pen\turntable\antenna1c-raw_frames-ts\g2o\zcpose-ahc-g2o.csv'
# # # m2fn='nothing'

# m1fn=r'E:\geo-cu399-pen\turntable\buddha1c-raw_frames-ts\g2o\kinfu_poses.csv'
# m2fn=r'E:\geo-cu399-pen\turntable\buddha1c-raw_frames-ts\g2o\zcpose-ahc-g2o.csv'
# # # m2fn=r'E:\geo-cu399-pen\turntable\budda1c-raw_frames-ts\cu&bdr\kinfu_poses-kf-t25-vs1.csv'

# # #temp
# # m1fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\zcpose-ahc-g2o.csv'

# # m1fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-kf-t10-tv2.csv'
# # m1fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-kf-t05-tv2.csv' #相比 kf-t10: 11.88mm
# m1fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu-tv21.5.8-t05.csv' #相比 kf-t10: 14.60mm, 相比 kf-t05: 22.21mm
# # m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t05-tv21.3.7.bef-vs06-e2c2mm.csv' #0.01mm, 相比于 21.5.8 约等于
# # m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu-t05-tv2-vs06-e2c2mm.csv' #0.98mm, 相比于 21.5.8 
# # m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-s2s[0.02,0.25,0]-tv2.csv' #144.46mm, f2f, td20mm 虽然误差很大, 但没飘走
# # m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-s2s[0.01,0.5,0]-tv2.csv' #153.50mm, f2f, td10mm, 类似上面
# # m1fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-s2s[0.01,0.5,1]-t05.csv' #248mm, 可能 f2m导致
# m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-s2s[0.01,0.5,0]-t10=BAD.csv' #1010mm, 超级差
# m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-s2s[0.01,0.5,1]-t02=BAD.csv' #1105mm 更差, 其实本来比上面好, 但是 f1072 突然严重飘走了
# # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'



# m1fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp123-tv11\kinfu_poses.csv'
# '都用 -123, 即 CuFusion, 看 tsdf -v2, -v11 对 odo 的影响:: ate=8.94mm'
# m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp123-tv2\kinfu_poses.csv'
# '都用 -tv11, 看 -123、 -1 区别:: ate=8.81mm; -1-tv11 与 -123-tv2 相比:: ate=13.14mm'
# m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp1-tv11\kinfu_poses.csv'
# '完全kf, -1-tv2:: ate=12.46mm'
# # m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp1-tv2\kinfu_poses.csv'

# m1fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp1-tv11\kinfu_poses.csv'
# m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp1-tv11\zcpose-ahc-g2o.csv' #ate=0
# m1fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp123-tv11\zcpose-ahc-g2o.csv'
# m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cu-every1\icp1-tv11\zcpose-ahc-g2o.csv' #ate=0

# # m1fn=r'D:\Users\zhangxaochen\Desktop\dbg\f2mkr-amp1\kinfu_poses.csv'
# # m2fn=r'D:\Users\zhangxaochen\Desktop\dbg\f2mkr-amp4\kinfu_poses.csv' #ate=5.73mm, 可忽略？
# '测试不成熟的 tv15, 表面重建噪声对于 pose 求解的影响: @2018-1-17 10:59:39'
# m1fn=r'D:\Users\zhangxaochen\Desktop\dbg\cu-t05-tv2\kinfu_poses.csv'
# m2fn=r'D:\Users\zhangxaochen\Desktop\dbg\cu-t25-tv15\kinfu_poses.csv' #ate=2mm, 

# '+++++++++++++++tv18.16 下, 测试: sdf2sdf 数据集 @2018-4-6 23:26:23'
# '1, Kinect_Teddy_Turntable'
# m1fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\markerboard7n.csv'
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\kf-vs05-t02-om\kinfu_poses.csv' #ate=11.2mm
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\kf-vs05-t05-om\kinfu_poses.csv' #ate=17.8mm
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\kf-vs05-t10\kinfu_poses.csv' #ate=35mm, 重建结果(看起来)好, 轨迹误差反而大, 为什么?
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\cu-vs05-t10-om\kinfu_poses.csv' #ate=8.85mm, 【说明 tv18.16 确实反过来促进了 odo】
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\cu-vs05-t02-om\kinfu_poses.csv' #ate=38.1mm, 
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\cu-vs05-t05-om\kinfu_poses.csv' #ate=17.0mm
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\cu-vs05-t15-om\kinfu_poses.csv' #ate=11.0mm
# m2fn=r'E:\sdf2sdf-data\Kinect_Teddy_Turntable\cu&bdr\bdr-vs05-t05-om\kinfu_poses.csv' #ate=47.2mm

# '2, Kinect_Tank_Turntable'
# m1fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\markerboard7n.csv'
# m2fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\cu&bdr\kf-vs05-t02-om\kinfu_poses.csv' #ate=35.6mm
# m2fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\cu&bdr\kf-vs05-t05-om\kinfu_poses.csv' #ate=28.9
# m2fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\cu&bdr\kf-vs05-t10-om\kinfu_poses.csv' #ate=29.3
# m2fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\cu&bdr\cu-vs05-t02-om\kinfu_poses.csv' #ate=37.6
# m2fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\cu&bdr\cu-vs05-t05-om\kinfu_poses.csv' #ate=33.8
# m2fn=r'E:\sdf2sdf-data\Kinect_Tank_Turntable\cu&bdr\cu-vs05-t10-om\kinfu_poses.csv' #ate=33.4

# '3, Kinect_Kenny_Turntable'
# m1fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\markerboard7n.csv'
# m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses.csv' #ate=490.132mm, 之前 4mm, 按2mm 更差
# m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses-d2mm-res2mm-bilat2.5.csv'
# m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses-d2mm-res2mm-bilat2.5-2.csv'

# m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses-tmp.csv'
# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses.csv'

# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses-res2mm.csv' #ate=490.132mm, 之前 4mm, 按2mm 更差
# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses=it40.csv' #ate=434.005mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses-intr-syn.csv' #327.38mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\s2s_poses=it20.csv' #434.615
# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\cu&bdr\kf-vs02-td02\kinfu_poses.csv' #149.232mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Kenny_Turntable\cu&bdr\kf-vs02-td05\kinfu_poses.csv' #89.420mm


# '4, Kinect_Bunny_Turntable'
# m1fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\markerboard7n.csv'
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses.csv' #173.793mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-d5mm-res2mm.csv' #192.570mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-d5mm-res4mm.csv' #148.015mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-d5mm-res4mm-bilat4.5.csv' #144.835mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-d5mm-res4mm-bilat2.5.csv' #120.981mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-bilat4.5-2.csv' #133.650 #仍是 -d5mm-res4mm, 但模板改一律 DTYPE, 
# m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-d5mm-res4mm-beta0.5-bilat[30,2.5].csv' #140.019
# m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-d5mm-res4mm-beta0.5-bilat[30,2.5]-omtm.csv' #104.208mm #-om -tm 加了桌子平面
# m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\s2s_poses-0.csv' #173.793mm

# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\cu&bdr\kf-vs02-td02\kinfu_poses.csv' #19.276mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\cu&bdr\kf-vs02-td02-om\kinfu_poses.csv' #29.716mm
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\cu&bdr\kf-vs02-td05\kinfu_poses.csv' #22.407
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\cu&bdr\kf-vs02-td05-om\kinfu_poses.csv' #36.219
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\cu&bdr\kf-vs05-td02-om\kinfu_poses.csv' #31.128
# # m2fn=r'E:\sdf2sdf-data\Kinect_Bunny_Turntable\cu&bdr\kf-vs05-td05-om\kinfu_poses.csv' #37.811
# m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses-s2s-f2f.csv' #f24, ate=82.2mm
# # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses-s2s-f2m.csv' #f24, ate=19.8mm
# # m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses-s2s-f2m-tm.csv' #f24, ate=13.1mm

# '5. Kinect_Bunny_Handheld'
# m1fn=r'E:\sdf2sdf-data\Kinect_Bunny_Handheld\markerboard7n.csv'
# m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'


# 'X, synthetic_circle_poses'
# # m1fn=r'E:\sdf2sdf-data\synthetic_circle_poses.csv'
# # # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-0.csv' #ate=32.459mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-bilat2.5.csv' #39.721mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-bilat4.5.csv' #25.182mm #上面三个是错的, 存档, 忽略
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-d5mm-res4mm-bilat0.csv' #66.812
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-d5mm-res4mm-beta0.5-bilat[20,2.5].csv' #27.320
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-d5mm-res4mm-beta0.5-bilat[15,2.5].csv' #23.301
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-d2mm-res2mm-beta0.2-bilat[15,2.5].csv' #123.068
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-d5mm-res4mm-bilat2.5.csv' #17.851
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\cu&bdr\kf-vs03-t02\kinfu_poses.csv' #ate=10.28
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\cu&bdr\kf-vs03-t05\kinfu_poses.csv' #ate=10.735
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\cu&bdr\kf-vs03-t10\kinfu_poses.csv' #ate=8.136
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\cu&bdr\kf-vs05-t02\kinfu_poses.csv' #ate=9.949
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Circle\s2s_poses-tmp.csv'

# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\s2s_poses-d2mm-res2mm-bilat0.csv' #ate=11.798
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\s2s_poses-d2mm-res4mm-bilat0.csv' #14.055
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\s2s_poses-d5mm-res4mm-bilat0.csv' #19.391
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\s2s_poses-d5mm-res4mm-bilat2.5.csv' #23.525
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\s2s_poses-d2mm-res2mm-bilat2.5.csv' #73.76
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\s2s_poses-d2mm-res2mm-bilat4.5.csv' #73.76 #跟 2.5 完全一样！！
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\cu&bdr\kf-vs03-t02\kinfu_poses.csv' #2.526mm
# # # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\cu&bdr\kf-vs03-t05\kinfu_poses.csv' #3.905
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Circle\cu&bdr\kf-vs03-t10\kinfu_poses.csv' #8.066

# 'Y, synthetic_wave_poses'
# # m1fn=r'E:\sdf2sdf-data\synthetic_wave_poses.csv'
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\s2s_poses-d5mm-res4mm-beta0.5-bilat[15,2.5].csv' #8.148mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\s2s_poses-d2mm-res2mm-beta0.2-bilat[15,2.5].csv' #8.093
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\s2s_poses-d5mm-res4mm-beta0.2-bilat[15,2.5].csv' #13.427
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\s2s_poses-d2mm-res4mm-bilat0.csv'#12.76
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\s2s_poses-d2mm-res2mm-bilat0.csv' #6.455
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\cu&bdr\kf-vs03-t02\kinfu_poses.csv' #4.578mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\cu&bdr\kf-vs03-t05\kinfu_poses.csv' #4.475
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\cu&bdr\kf-vs03-t10\kinfu_poses.csv' #6.737
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\cu&bdr\kf-vs05-t02\kinfu_poses.csv' #5.955
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Bunny_Wave\s2s_poses-tmp.csv'

# # m2fn=r'E:\sdf2sdf-data\Synthetic_Tank_Wave\cu&bdr\kf-vs03-t02=BAD\kinfu_poses.csv'
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Tank_Wave\cu&bdr\kf-vs03-t05=BAD\kinfu_poses.csv'

# # m2fn=r'E:\sdf2sdf-data\Synthetic_Leopard_Wave\cu&bdr\kf-vs03-t02\kinfu_poses.csv' #6.756mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Leopard_Wave\cu&bdr\kf-vs03-t05\kinfu_poses.csv' #8.453
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Leopard_Wave\cu&bdr\kf-vs03-t10\kinfu_poses.csv' #11.077
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Leopard_Wave\cu&bdr\kf-vs05-t02\kinfu_poses.csv' #5.829

# # m2fn=r'E:\sdf2sdf-data\Synthetic_Teddy_Wave\s2s_poses.csv' #282.644mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Teddy_Wave\cu&bdr\kf-vs03-t02\kinfu_poses.csv' #1.474mm
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Teddy_Wave\cu&bdr\kf-vs03-t05\kinfu_poses.csv' #1.511
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Teddy_Wave\cu&bdr\kf-vs03-t10\kinfu_poses.csv' #1.670
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Teddy_Wave\cu&bdr\kf-vs05-t02\kinfu_poses.csv' #1.550

# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Wave\s2s_poses.csv' #277.019
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Wave\cu&bdr\kf-vs03-t02\kinfu_poses.csv' #6.934
# # m2fn=r'E:\sdf2sdf-data\Synthetic_Kenny_Wave\cu&bdr\kf-vs05-t02\kinfu_poses.csv' #8.914

# m1fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-cu-t05-tv18-vs0.7.csv'
# # m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-s2s-f2m-t10-vs0.7.csv' #f2m, ate=
# m2fn=r'C:\geo-cu399-pen\long2-jibei-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-s2s-f2f-t10-vs0.7.csv'

# m1fn=r'C:\geo-cu399-pen\long1-jlong-c-raw_frames-ts\cuOdo&bdr-2\kinfu_poses-cu-t05-vs0.5-tv18.18.csv'
# m2fn=r'C:\geo-cu399-pen\long1-jlong-c-raw_frames-ts\cuOdo&bdr-2\kinfu_poses-kf-t05-vs0.6.csv'

m1fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.3.7.bef-vs06-t05-e2c2mm.csv'
m1fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm.csv'
# m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses.csv'
m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP2-743.csv' #1.03mm
# m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP4.2=BAD.csv'#3.33mm
# m2fn=r'C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP4.3=GOOD.csv'#1.2mm
m2fn=r"C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP4.4.csv"#1.78mm
m2fn=r"C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP4.4.2=BAD.csv"#8.69mm
m2fn=r"C:\geo-cu399-pen\long4-changjing-c-raw_frames-ts\cu&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP4.4.3.csv"#2.13mm

m1fn=r"C:\geo-cu399-pen\long3-yilong-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-cu2-tv21.3.7.bf-GOOD.csv"
m1fn=r"C:\geo-cu399-pen\long3-yilong-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP4.4.3.csv"
m2fn=r"C:\geo-cu399-pen\long3-yilong-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP(1043).csv"
m2fn=r"C:\geo-cu399-pen\long3-yilong-c-raw_frames-ts\cuOdo&bdr\kinfu_poses-cu2-tv21.5.8+21.3.7.bef-vs06-t05-e2c2mm-step2-ICP(1043)-noE2C.csv"
# m2fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\kinfu_poses.csv'
# m2fn=r"E:\Github\StanfordPCL\_build.vc10.x64\bin\zcpose-ahc-g2o.csv"

close('all')
m1tvecs, m1rmats, m2tvecs, m2rmats, ate = diffCsvs(m1fn, m2fn, rng, assoFn=assoFn) #包括了plot 两组tvecs、以及ATE

# figure()
# subplot(312)
# title('m1&m2-tvecs')
# m1ts = plot(m1tvecs)
# m2ts = plot(m2tvecs)
# legend(m1ts+m2ts, ['m1x', 'm1y', 'm1z', 'm2x', 'm2y', 'm2z'])

figure()
# subplot(313)
# xlim(-1, 1700)
# ylim(0, 0.015)
title('m1&m2-tXZ')
cutNum=None#26#+1
t1=m1tvecs[:cutNum]
t2=m2tvecs[:cutNum]
axx,axy=0,2 #选用哪两个轴绘制
# pts1 = scatter(t1[:, (axx,axy)], c='r') #scatter报错, plot也逻辑错(画成两条线)
pts1 = scatter(t1[:, axx], t1[:, axy], c='r') #先画, 在底层, 所以用大点
pts2 = scatter(t2[:, axx], t2[:, axy], c='g', marker='x')
# legend([pts1,pts2], list('12'), loc='lower right')
legend([pts1,pts2], ['GT', 'our'], loc='lower right')
annotate('start', t1[0, (axx,axy)])
annotate('end', t1[-1, (axx,axy)])
savefig('foo.png')

--diffCsvs(), len(m1), len(m2), minLen= 1622 1622 1622
--mean-abs-diff(各列): [ 0.01355856  0.01549365  0.00917172  0.00308925  0.00962635  0.00423586
  0.00842895]
ATE-RMSE: 0.0256283898885


In [25]:
idx=32
t1i, t2i = t1[idx], t2[idx]
t12i = t1i - t2i
print t1i, t2i, t12i, norm(t12i)

[ 0.189728 -0.239364 -0.278029] [ 0.20043  -0.230387 -0.260669] [-0.010702 -0.008977 -0.01736 ] 0.0222820316174


In [ ]:
(-2**31)

In [64]:
m1fn

'C:\\geo-cu399-pen\\long2-jibei-c-raw_frames-ts\\cu-every1\\icp123-tv11\\kinfu_poses.csv'

In [8]:
atekf=ate

In [10]:
atebdr=ate

In [11]:
ateef=ate

In [12]:
atecu=ate

In [13]:
'''
2017-7-23 16:30:09
按最短序列长度截断, 各方法 ATE 绘制在一幅图中
'''
cla()
minLen = min(len(atekf), len(atebdr), len(ateef), len(atecu))
print 'minLen:', minLen
atekf = atekf[:minLen]
atebdr = atebdr[:minLen]
ateef = ateef[:minLen]
atecu = atecu[:minLen]
ates = vstack((atekf, atebdr, ateef, atecu)).T
ates *= 1e3
# plot(ates, '.-')
plot(ates)
# ylim(0, 0.02)
xlabel('time (k)')
ylabel('Absolute Trajectory Error (ATE) (mm)')
legend(['KinectFusion',
        'Zhou et al.',
        'ElasticFusion',
        'Ours'
       ])

minLen: 663


In [92]:
ateef.shape

(661L,)

In [93]:
atekf.shape

(661L,)

In [6]:
'单单绘制 m1fn(一般是GTruth), 观察tvec'
# m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep0\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep1\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep2\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep3\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep4\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep5\syn_pose.csv'

m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep8\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep9\syn_pose.csv'
m1fn=r'E:\Github\pcl\zctest\_build.vc10\synDep10\syn_pose.csv'

m1 = loadtxt(m1fn, delimiter=',')
m1tvecs = m1[:, :3]
close()
figure(figsize=(6,6))
axx,axy=0,1 #选用哪两个轴绘制
# pts1 = scatter(m1tvecs[:, 0], m1tvecs[:, 1], c='r')
pts1 = plot(m1tvecs[:, axx], m1tvecs[:, axy], c='r', ls='-')
annotate('start', m1tvecs[0, (axx,axy)])
annotate('end', m1tvecs[-1, (axx,axy)])
savefig('foo2.png')

In [17]:
'绘制深度图, 放到 draft.docx 里 @2017-8-6 22:27:20'

from glob import glob
import os
fold=r'E:\Github\pcl\zctest\_build.vc10\synDep9\depth'
fold=r'E:\geo-cu399-pen\turntable\bunny1c-raw_frames-ts\depth'
fold=r'E:\geo-cu399-pen\turntable\owl1c-raw_frames-ts\depth'
fold=r'E:\geo-cu399-pen\turntable\wingedcat1c-raw_frames-ts\depth'
fold=r'E:\geo-cu399-pen\turntable\budda1c-raw_frames-ts\depth'
fold=r'E:\geo-cu399-pen\turntable\tooth1c-raw_frames-ts\depth'
fold=r'E:\geo-cu399-pen\turntable\cup6down1c-raw_frames-ts\depth'
fns=glob(os.path.join(fold,'*.png'))
fns=sorted(fns, key=lambda fn: float(os.path.splitext(os.path.basename(fn))[0]))
fn=fns[0]
img=imread(fn)
cmap='gray'
#cmap='Greys'
imshow(img, cmap=cmap)
imsave('shitt.png', img, cmap=cmap)

In [297]:
'''
2016-12-13 11:27:31
---------------ahc、pcc 如何确定谁更精确? 
【rgb-cb-gt.csv 不可信】, 故测试: 各自收尾帧转到世界坐标系，看各自尾帧与首帧重叠度好坏
首帧只要一个即可，因为首帧必然完全重叠

【此cell目标】： 将 m2 转换到 m1 坐标系, 使 m2[0]==m1[0], 并输出 m2[-1]
    【暂放弃 2016-12-13 11:37:15】 因为目前 ahc 作 m2 时, c++ 程序里已经做过坐标系对齐了 (默认 (1.5,1.5,-0.3, 1,0,0,0))
没放弃吧？↑-为啥这么说来？记不清了，但肯定对应报告：《顶角视角ahc-kf-pcc三种方法对比结果草稿.docx》
'''


def printT44(pose7):
    'pose7: t3(m)+r4(qua) array, r4~wxyz序, 注意若用 tf 则要 xyzw 序'
    t = pose7[:3]
    r = pose7[3:]
    qua = hstack((r[1:], r[0]))
    T = tf.q2m(qua) #4x4
    T[:3, 3] = t # 不必 t.T, 因为 .T 也没用
    print T
    pass

m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\zcpose.kf.csv' #cb-gt:13.3mm
# m1fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\zcpose.pcc.csv' #cb-gt:12.7mm; kf:1mm; 
m2fn=r'D:\Users\zhangxaochen\Documents\geo-cube399\cb5x8-cu399c-raw_frames-ts2\pose-ahc.csv' #cb-gt:12.9mm; pcc:4.9mm; kf:5.1mm

#拷贝自 cell: diffCsvs
m1 = loadtxt(m1fn, delimiter=',')[:, :7] #本来 getExtrs.ipynb 生成的 rgb...csv 是8列, 但是目前感觉第8列 ts 没用 #2016-12-12 01:21:38
m2 = loadtxt(m2fn, delimiter=',')[:, :7]
row10 = m1[0]
row1n = m1[-1]
printT44(m1[0])
printT44(m1[-1])
printT44(m2[0])
printT44(m2[-1])


[[ 1.   0.   0.   1.5]
 [ 0.   1.   0.   1.5]
 [ 0.   0.   1.  -0.3]
 [ 0.   0.   0.   1. ]]
[[ 0.88124295  0.34026881 -0.32806707  1.85131   ]
 [-0.33783451  0.93886705  0.06630623  1.42647   ]
 [ 0.3305733   0.05240048  0.94232451 -0.240058  ]
 [ 0.          0.          0.          1.        ]]
[[ 1.   0.   0.   1.5]
 [ 0.   1.   0.   1.5]
 [ 0.   0.   1.  -0.3]
 [ 0.   0.   0.   1. ]]
[[ 0.88264002  0.33567351 -0.3290439   1.85255   ]
 [-0.33513793  0.9402438   0.06020102  1.43342   ]
 [ 0.32958938  0.05713926  0.94239373 -0.241384  ]
 [ 0.          0.          0.          1.        ]]


In [35]:
'''
2016-12-13 11:24:49
【放弃此 cell】 因为没考虑同步问题, 已修改 diffCsvs 加入 assoFn 参数, 考虑 TUM 式同步

2016-12-8 16:35:46
---------------在立方体真实数据上, 只扫描一个顶角, 三种结果对比: 
基本假设: dmap, color-img 已经同步过;
1. GT: matlab-cb-外定标 (cb-gt.csv, 有无效全零行); 坐标系: cb->cam
2. ahc/peac 定位立方体;  有无效全零行; 坐标系: cam_i->cam_0(第一个有效帧, 未必是整体序列第一帧);
3. kf/pcc, ICP 配准;  有无效全零行; 坐标系: cam_i->cam_0(整体第一帧)
'''
m1fn=r'E:\oni_data\cb-cube-corner20161205-raw_frames\cb-gt.csv'
# m2fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose-cb-cube-corner20161205-raw_frames.csv' #这命名太乱了
m2fn=r'E:\oni_data\cb-cube-corner20161205-raw_frames\pose-ahc.csv'
m3fn=r'E:\Github\pcl\_build.vc10\bin\cb-cube-corner20161205-raw_frames-kf\zcpose.csv'

'1. 首先获取 cb-gt.csv 中第一有效帧, 这是 cb 的全局坐标系/参考帧'
m1=loadtxt(m1fn, delimiter=',')
firstValidId=-1
for i, row in enumerate(m1):
    if not np.allclose(row, 0):
        firstValidId = i #第一次有效帧, 全局参考帧
        break
    pass

'2. '
lines=plotTvecs(m1fn)
legend(lines, list('xyz'))
figure()
lines=plotTvecs(m2fn)
legend(lines, list('xyz'))


In [124]:
'''
2017-1-2 23:00:26
在同一世界坐标系中, 3D 绘制立方体各个顶角, 观察一致性
'''

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
# clf() #不好
# cla() #也不太好
ax = fig.gca(projection='3d')

'05'
fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\cube-crnr-axes-pcc.csv'
# fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\cube-crnr-axes-kf.csv' #几乎相同
fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\cube-crnr-axes-ahc.csv'
'↑-上三个结果图备份在：http://imgur.com/a/mRB4Z'
fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\05f80c-raw_frames-ts\output-cuicp\cube-crnr-axes.csv'
fn=r'E:\Github\StanfordPCL\_build.vc10.x64\bin\cube-crnr-axes.csv'

'10'
# fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\10f80c-raw_frames-ts\cube-crnr-axes-kf.csv'
# fn=r'D:\Users\zhangxaochen\Documents\geo-cu399-cb\10f80c-raw_frames-ts\cube-crnr-axes-ahc.csv'

m1=loadtxt(fn, delimiter=',')
print m1.shape

lines=[]
scale = 5.251 #轴长, 量纲:米(m)
colors=list('rgb')
# m1=m1[:444]
# for i, row in enumerate(m1):
rng=range(10)+range(233,255)+range(333, 355)+range(444,466)+range(666,699)
rng=slice(None)
rng=slice(None,None,4)
# rng=slice(0,40,2)
for i, row in enumerate(m1[rng]):
#     print 'i, t,q:', i, row
    t3 = row[:3]
    q4 = row[3:]
    rmat33 = tf.q2m([q4[1], q4[2], q4[3], q4[0]])[:3, :3]
#     print repr(t3)
#     print repr(q4)
#     print repr(rmat33)
    
    '每一列对应 xyz 一条轴线段:'
    for j in range(3):
        axj = rmat33[:, j]
        lines.extend(
            plot([t3[0]- scale*axj[0], t3[0]+ scale*axj[0]],
             [t3[1]- scale*axj[1], t3[1]+ scale*axj[1]],
             [t3[2]- scale*axj[2], t3[2]+ scale*axj[2]], colors[j])
        )
        pass
    pass
        

(1049L, 7L)


In [115]:
m1[rng]

array([ 0.233572, -0.362743])

In [93]:
tf.qmul(tf.qinv(array([ 0.591489,  0.337254, -0.639585,  0.356836])), array([ 0.59055 ,  0.338056, -0.640616,  0.355781]))

array([  1.23713545e-04,  -5.68409804e-04,  -1.83371504e-03,
         9.99998020e-01])

In [92]:
tf.qmul?

In [87]:
print rmat33

[[-0.07553714 -0.85200337 -0.5180583 ]
 [-0.01293449  0.52033614 -0.85386357]
 [ 0.9970591  -0.05779759 -0.05032493]]


In [58]:
'''
2017-1-3 17:29:59
上面cell 注掉 fig = plt.figure() 时, 希望在同一 fig 中调试观察不同实验

'''
print 'len(lines):', len(lines)

for ln in lines:
    ln.remove()

len(lines): 318


[//]:#(![](http://i.imgur.com/EAyLF3v.png))
<img src="http://i.imgur.com/EAyLF3v.png" width="444">

[//]:#(![](http://i.imgur.com/6Bc9aIy.png))
<img src="http://i.imgur.com/6Bc9aIy.png" width="444">

In [40]:
[0]+range(3)

[0, 0, 1, 2]

In [39]:
m1[0, range(3)]

array([ 1.61056 ,  1.72318 ,  0.719515])

In [34]:
m1[[0,3]]

array([[ 1.61056 ,  1.72318 ,  0.719515,  0.132603, -0.469627, -0.290332,
         0.823149],
       [ 1.62505 ,  1.52136 ,  0.568193,  0.130999, -0.472137, -0.289828,
         0.822147]])

In [11]:
tf.q2m?

In [ ]:
'''
2016-5-24 09:31:55
---------------(txyz+qwxyz) 7列转为 (txyz+M11~33) 12列,
用于测试以 kinfu 输出的姿态重新跑, 在 -icp0 -eroRad K 的时候的效果
'''

fin = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-pcc\zcpose.csv' #pcc
fout = r'zcpose-pcc-c12.txt'

fin = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-t0(-0.3)\zcpose.csv' #kf.orig
fout = r'zcpose-kf-c12.txt'

m = loadtxt(fin, delimiter=',')
tvecs = m[:, :3]
rvecs = m[:, 3:]
rmats = np.asarray([tf.q2m([q[1], q[2], q[3], q[0]])[:3, :3].ravel() for q in rvecs]) #4x4->3x3
# np.hstack((rmats[0::3], rmats[1::3], rmats[2::3]) #这样不行, 上面改成 .ravel() 就够了
'另外, 目前 synRt...txt 文件里存的 t 是毫米'
tvecs *= 1000
mout = np.hstack((tvecs, rmats))
savetxt(fout, mout, '%.12g')


In [5]:
'misc test, 2016-9-29 16:58:27'
fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\zcpose.csv'
fn=r'E:\ABOUT-C++\libs\peac\_build.vc10\pose.csv'
lines=plotTvecs(fn)
legend(lines, list('xyz'))

In [18]:
m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-t0(-0.3)\zcpose.csv' #kf.orig
m1fn = r'E:\Github\pcl\_build.vc10\bin\cb5x8-cu399c-newIntr-reg=2-g2c\zcpose.csv' #-g2c
m1mat = loadtxt(m1fn, delimiter=',')
m1tvecs = m1mat[:, :3]
print 'len m1tvecs:', len(m1tvecs)
plot(m1tvecs)
legend(list('xyz'))

len m1tvecs: 110


In [21]:
tf.e2m?

In [3]:
'测试 pcl_kinfu_app 使用 -r 参数与不用时的差异'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slope-7x10-264-r-2.csv'
pose_mat = loadtxt(pose_file, delimiter=',')
tvecsDepth = pose_mat[:, :3]
lns = plot(tvecsDepth)
# legend(lns, list('xyz'))

pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slope-7x10-285.csv'
pose_mat = loadtxt(pose_file, delimiter=',')
tvecsDepth = pose_mat[:, :3]
lns2 = plot(tvecsDepth)
legend(lns+lns2, list('xyzabc'))

# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slope-7x10-T0=0.csv'
# pose_mat = loadtxt(pose_file, delimiter=',')
# tvecsDepth = pose_mat[:, :3]+(1.5, 1.5, -0.3)
# plot(tvecsDepth)


In [6]:
# generated by pcl_kinfu_app & OpencvCalib:
'3x4, t2b-refine'
'---------------1'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-sz35.8-r2l-t2b-refine.csv'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_4x3\smt-d100-sz35.8-r2l-t2b-refine_calib\extrinsics.xml'

'horizon-move-part1, c-r(center->right)'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-sz35.8-horizon-move-part1.csv'
'---------------2'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_4x3\smt-d100-sz35.8-horizon-move-part1_calib\extrinsics-3x4.xml'

'---------------3'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_4x3\smt-d100-sz35.8-horizon-move-part1_calib\extrinsics-4x3.xml'

'vertical, c-t-b-c'
'---------------4'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-sz35.8-vertical-move.csv'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_4x3\smt-d100-sz35.8-vertical-move_calib\extrinsics-4x3.xml'

'zrot-7x8, 先逆时针'
'---------------5'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-zrot-7x8.csv'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_7x8\smt-d100-zrot-7x8_calib\extrinsics.xml'

'xyzrot-7x8-3axes, xyz先后转动'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-xyzrot-7x8-3axes.csv'
'608 frmaes:'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_7x8\smt-d100-xyzrot-7x8-3axes_calib\extrinsics0.xml'
'1034 frames:'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_7x8\smt-d100-xyzrot-7x8-3axes_calib\extrinsics1.xml'
'---------------6'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_7x8\smt-d100-xyzrot-7x8-3axes_calib\extrinsics1-22.3.xml'

'xyztrans, 7x8, 测试平移'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-xyztrans-7x8-refine.csv'
'---------------7'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_7x8\smt-d100-xyztrans-7x8-refine_calib\extrinsics.xml'

'bigboard-d130-7x10.oni, 大板三轴乱动'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-d130-7x10.csv'
'---------------8'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI1.3_data_registration_7x9\bigboard-d130-7x10_calib\extrinsics.xml'

'bigboard-slope-7x10, 大板子斜着转'
'---------------9'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slope-7x10-264.csv'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slope-7x10-285-T0=0.csv'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slope-7x10-285.csv'
# extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\bigboard-slope-7x10_calib\extrinsics-22.3.xml'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openni132_registration_7x10\bigboard-slope-7x10_calib\extrinsics-50.xml'

'smallboard, horizon, c-r-l-c, 7x8'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smallboard-d100-7x8.csv'
# extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\smallboard-d100-7x8_calib\extrinsics-8x7.xml'

'horizon-move, c-r-l-c'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-sz35.8-horizon-move.csv'
# extrinsics_xml = r"E:\GitHub\OpenniCeshi\data\openNI132_data_registration_4x3\smt-d100-sz35.8-horizon-move_calib\extrinsics-3x4.xml"
# extrinsics_xml = r"E:\GitHub\OpenniCeshi\data\openNI132_data_registration_4x3\smt-d100-sz35.8-horizon-move_calib\extrinsics-4x3.xml"

'smt-d100-7x8-move-pause, A4小纸'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-smt-d100-7x8-move-pause.csv'
# extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openNI132_data_registration_7x8\smt-d100-7x8-move-pause_calib\extrinsics-1456.xml'

'bigboard-7x10-move-pause'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause.csv'
extrinsics_xml = r'E:\GitHub\OpenniCeshi\data\openni132_registration_7x10\bigboard-7x10-move-pause_calib\extrinsics-1846.xml'

'bigboard-7x10-move-pause-f100~f990.oni'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-shit90.csv'
# extrinsics_xml = r'E:\oni_data\openni132_registration_7x10\shit90_calib\extrinsics.xml'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990.csv' #total: 89
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-raw_frames.csv' #total: 91
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-raw_frames-90-urf.csv' #total: 90
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-89-cv_extr.csv'
'-urf, -cv_extr, 90, 此文件确实与 extrinsics.xml 曲线重合； rmse == 0'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-cv_extr.csv'
'-cv_extr_hint, 用 extrinsics.xml 做初值, 帧数太少，看不出什么； 曲线与 -urf-90 几乎完全重合（数值看起来不同）'
pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-cv_extr_hint.csv'
'之前一直带着 -r -ic, 这里去掉它们'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-no-r-ic.csv'
extrinsics_xml = r'E:\oni_data\openni132_registration_7x10\bigboard-7x10-move-pause-f100~f990_calib\extrinsics.xml'


'2014/12/18, 之前 oni 文件在 pcl 处理有掉帧，不确定是不是 niviewer 的问题'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-move_around-2.csv'
# extrinsics_xml = r'E:\oni_data\openni132_registration_7x10\bigboard-move_around-2_calib\extrinsics.xml'

# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slow-refine.csv'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slow-refine-2061-urf.csv'
# pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slow-refine-2056-no-urf.csv'
# # pose_file = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slow-refine-raw_frames-2.csv'
# extrinsics_xml = r'E:\oni_data\openni132_registration_7x10\bigboard-slow-refine_calib\extrinsics.xml'


'2014/12/18, \xe4\xb9\x8b\xe5\x89\x8d oni \xe6\x96\x87\xe4\xbb\xb6\xe5\x9c\xa8 pcl \xe5\xa4\x84\xe7\x90\x86\xe6\x9c\x89\xe6\x8e\x89\xe5\xb8\xa7\xef\xbc\x8c\xe4\xb8\x8d\xe7\xa1\xae\xe5\xae\x9a\xe6\x98\xaf\xe4\xb8\x8d\xe6\x98\xaf niviewer \xe7\x9a\x84\xe9\x97\xae\xe9\xa2\x98'

In [7]:
print os.path.isfile(extrinsics_xml)
print os.path.isfile(pose_file)

True
True


In [3]:
'以下说明， cv_extr 做初值没有任何提升'

# fn = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-cv_extr.csv'
# plotTvecs(fn)
# fn = r'E:\oni_data\openni132_registration_7x10\bigboard-7x10-move-pause-f100~f990_calib\extrinsics.xml'

'以下说明， cv_extr 做初值没有任何提升'
'-no-r-ic'
# fn = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-no-r-ic.csv'
# plotRvecs(fn)
# fn = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-cv_extr_hint-no-r-ic.csv'
# plotRvecs(fn)

'-r-ic'
fn = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90.csv'
plotRvecs(fn)
fn_hint = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-cv_extr_hint.csv'
plotRvecs(fn_hint)
print 'diffCsvs(fn, fn_hint)', diffCsvs(fn, fn_hint)

'视频 2061 帧'
fn = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slow-refine-2061-urf.csv'
plotRvecs(fn)
fn_hint = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-slow-refine-2061-urf-cv_extr_hint.csv'
plotRvecs(fn_hint)
print 'diffCsvs(fn, fn_hint)', diffCsvs(fn, fn_hint)

# fn = r'E:\ABOUT C++\libs\pcl-master-6d0343d1b7\_build\gpu\kinfu\tools\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90-cv_extr_hint.csv'
# plotTvecs(fn)


IOError: [Errno 2] No such file or directory: 'E:\\ABOUT C++\\libs\\pcl-master-6d0343d1b7\\_build\\gpu\\kinfu\\tools\\pose_file-bigboard-7x10-move-pause-f100~f990-urf-90.csv'

In [55]:
'---------------处理 pcl_kinfu_app 生成的 csv'
pose_mat = loadtxt(pose_file, delimiter=',')
tvecsDepth = pose_mat[:, :3]
rvecsDepth = pose_mat[:, 3:]
eulerDepth = np.asarray([tf.q2e([q[1], q[2], q[3], q[0]], axes='rxyz') for q in rvecsDepth])
matDepth = np.asarray([tf.q2m([q[1], q[2], q[3], q[0]]) for q in rvecsDepth])
print rvecsDepth.shape
# print rvecsDepth

'接上个 cell'
print eulerDepth.shape
# print eulerDepth

'接上个 cell'
# xs = range(eulerDepth.size)

# figure('111')
# plot(eulerDepth[:, 0], 'r')
# plot(eulerDepth[:, 1], 'g')
# plot(eulerDepth[:, 2], 'b')

# figure('222')
fig = figure('1-eulerDepth-bgr')
plot(eulerDepth, ) #与 111 相反， b,g,r 顺序
savefig(fig.canvas.get_window_title())


'---------------处理 OpencvCalib 生成的 extrinsics.xml'
from lxml import etree
psr = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(extrinsics_xml, parser=psr)
rt = tree.getroot()

# Rodrigues 向量：
rvecsColor = []
tvecsColor = []
# for iframe in rt:
for i, iframe in enumerate(rt):
    rvecsColor.append([float(num) for num in iframe.xpath('r/data')[0].text.split()])
#     rvecsColor.append(map(float, iframe.xpath('r/data')[0].text.split())) # python 2, 3 不兼容， 所以不用
    tvecsColor.append([float(num) for num in iframe.xpath('t/data')[0].text.split()])
    pass

rvecsColor = np.asarray(rvecsColor)
tvecsColor = np.asarray(tvecsColor)
# rvecsColor

# dst, _ = cv2.Rodrigues(rvecsColor[0])
# eulerColor = np.asarray([tf.euler_from_matrix(cv2.Rodrigues(rod)[0]) for rod in rvecsColor]) #good √
# R0 = cv2.Rodrigues(rvecsColor[0])[0].T
'---------------'
eulerColor = []
eulerColor2 = [] # 'R0.T *' 处理过的
'camera -> world 坐标系变换：'
matColor = np.asarray([cv2.Rodrigues(rod)[0].T for rod in rvecsColor]) #---------------有没有 .T ? 应该有
matColor2 = []
R0 = matColor[0]
# for rod in rvecsColor:
#     Ri = cv2.Rodrigues(rod)[0].T
for Ri in matColor:
    eulerColor.append(tf.m2e(Ri, axes='rxyz'))
    '--2:'
#     newRi = np.dot(R0.T, R0.T).dot(Ri).dot(R0) #貌似错了
    
    '--1:'
#     newRi = np.dot(R0.T, Ri).dot(R0.T).dot(R0) #---------------仅仅 R0.T * Ri， （全 .T 之后， 见上 line58)
    newRi = np.dot(R0.T, Ri) #---------------??
#     newRi = np.dot(Ri, R0.T) #---------------×
    eulerColor2.append(tf.m2e(newRi, axes='rxyz')) #R0.T * (R0.T * Ri) * R0
#     eulerColor2.append(tf.euler_from_matrix(np.dot(Ri, R0.T), axes='rxyz')) #尝试右乘 ×
    matColor2.append(newRi)
    pass

matColor2 = np.asarray(matColor2)
matDiff = np.asarray([np.dot(matDepth[i][:3, :3].T, matColor2[i]) for i in range(min(matDepth.shape[0], matColor2.shape[0]))])

eulerColor = np.asarray(eulerColor)
eulerColor2 = np.asarray(eulerColor2)

'---------------一些比较测试'
#333
fig = figure('2-eulerColor-orig')
plot(eulerColor)
savefig(fig.canvas.get_window_title())

#444
fig = figure('3-eulerColor-newRi')
plot(eulerColor2)
savefig(fig.canvas.get_window_title())

fig = figure('4-euler-depth-color')
plot(eulerDepth)
plot(eulerColor2)
savefig(fig.canvas.get_window_title())

fig = figure('5-matDiff-main-diagonal')
plot(matDiff[:,0,0])
plot(matDiff[:,1,1])
plot(matDiff[:,2,2])
savefig(fig.canvas.get_window_title())

fig = figure('6-matDiff-9-elems')
plot(matDiff.reshape((matDiff.shape[0], 9)))
savefig(fig.canvas.get_window_title())

'matDiff 转成轴角表示'
m = matDiff[0]
fig = figure('7-matDiff-rodrigues-angle')
plot([np.linalg.norm(cv2.Rodrigues(m)[0]) for m in matDiff])
savefig(fig.canvas.get_window_title())

# 'm-e, 求行列式, 应该结果接近 0 @孙国飞。已否定'
# fig = figure('matDiff-eye-det')
# plot([np.linalg.det(m-np.eye(3)) for m in matDiff ])
# savefig(fig.canvas.get_window_title())

fig = figure('8-tvecs-depth-color')
plot(tvecsDepth)
plot(tvecsColor/1000)
savefig(fig.canvas.get_window_title())

tvecsColorOrig = tvecsColor

'''
---------------测试对比 ta, tb ：

对于 cv, color, extrinsics.xml:
先转到物体旋转：
Ri' := Ri.T
ti' := -Ri.T * ti
t0' = -R0.T * t0

ta = Rab.T * (Rb * tab + tb - tab)
'''
tvecsColor = tvecsColorOrig / 1000
t0c = tvecsColor[0]
t0d = tvecsDepth[0]
deltatColor = np.asarray([np.dot(matColor[i], t0c-tvecsColor[i]) for i in range(tvecsColor.shape[0])])
tbColor = deltatColor
tCV2PCL = np.asarray([(np.dot(-R0.T, matColor[i]).dot(tvecsColor[i]) + t0c + t0d) for i in range(tvecsColor.shape[0])]) #√

'''
对于 pcl_kinfu_app, depth, pose_file.csv
ta = ΔtiDepth = tid - Rid * t0d
'''
deltatDepth = tvecsDepth - np.asarray([np.dot(matDepth[i][:3, :3], t0d) for i in range(tvecsDepth.shape[0])])
taDepth = deltatDepth

fig = figure('9-tCV2PCL-deltatDepth')
plot(tCV2PCL)
plot(deltatDepth)
savefig(fig.canvas.get_window_title())

fig = figure('10-tvecsDepth-tCV2PCL')
plot(tvecsDepth)
plot(tCV2PCL)
savefig(fig.canvas.get_window_title())

'对比 pcl, opencv 的 r, t（转换后），求 RMSEe, RMSEt'
ne = min(eulerColor2.shape[0], eulerDepth.shape[0])
ediff = eulerColor2[:ne] - eulerDepth[:ne]
rmsee = np.sqrt(np.square(ediff).mean(axis=0))

nt = min(tCV2PCL.shape[0], tvecsDepth.shape[0])
tdiff = tCV2PCL[:nt] - tvecsDepth[:nt]
rmset = np.sqrt(np.square(tdiff).mean(axis=0))

print 'rmsee, rmset:'
print '%.3f, %.3f, %.3f'%tuple(rmsee)
print '%.3f, %.3f, %.3f'%tuple(rmset)
# print ', '.join(['%.3f']*rmsee.size)%tuple(rmsee) #good, √


(90L, 4L)
(90L, 3L)
rmsee, rmset:
0.007, 0.010, 0.015
0.011, 0.007, 0.011


In [49]:
def printErrorOnIdx(error, indices):
    ur'打印error列表在 indices 序号上的值'
    print np.array2string(error[indices], precision=3)
    pass

absEdiff = abs(ediff)
print absEdiff.shape

'某 ids 几个序号 R 误差'
# ids = [22, 45, 67]
ids = [515, 1030, 1545]
printErrorOnIdx(absEdiff, ids)

'前10帧 R 误差绝对值'
# printErrorOnIdx(absEdiff, range(10))

'ids T误差'
printErrorOnIdx(abs(tdiff), ids)


(2061L, 3L)
[[ 0.016  0.016  0.035]
 [ 0.     0.005  0.008]
 [ 0.     0.046  0.028]]
[[ 0.018  0.032  0.015]
 [ 0.002  0.014  0.014]
 [ 0.023  0.004  0.022]]


In [17]:
a = rmsee
', '.join(['%.3f']*a.size)%tuple(a)

'0.006, 0.010, 0.015'

In [269]:
e1 = np.array([10,20,30])*np.pi/180
m1 = tf.e2m(*e1, axes='rxyz')
np.asarray(tf.m2e(m1, axes='rxyz'))*180/np.pi

e2 = -e1
m2 = tf.e2m(*e2[::-1], axes='rxyz')
print 'e1:', e1
print 'm1:', m1
print 'e2:', e2
print 'm2:', m2

print 'e2::-1,',e2[::-1]

e1: [ 0.17453293  0.34906585  0.52359878]
m1: [[ 0.81379768 -0.46984631  0.34202014  0.        ]
 [ 0.54383814  0.82317294 -0.16317591  0.        ]
 [-0.20487413  0.31879578  0.92541658  0.        ]
 [ 0.          0.          0.          1.        ]]
e2: [-0.17453293 -0.34906585 -0.52359878]
m2: [[ 0.92541658  0.16317591 -0.34202014  0.        ]
 [ 0.01802831  0.88256412  0.46984631  0.        ]
 [ 0.37852231 -0.44096961  0.81379768  0.        ]
 [ 0.          0.          0.          1.        ]]
e2::-1, [-0.52359878 -0.34906585 -0.17453293]


In [173]:
'测试 xyz -> yzx 坐标轴变换, 是否效果仅仅是交换坐标轴？ 不是！'

figure('xyz')
eulerDepth = np.asarray([tf.q2e([q[1], q[2], q[3], q[0]], axes='rxyz') for q in rvecsDepth])
plot(eulerDepth, )

figure('yzx')
eulerDepth = np.asarray([tf.q2e([q[1], q[2], q[3], q[0]], axes='ryzx') for q in rvecsDepth])
plot(eulerDepth, )


In [5]:
'some tests:'
# 1. from doc:
a = tf.euler_from_quaternion([0.06146124, 0, 0, 0.99810947])
print type(a)
print np.allclose(a, [0.123, 0, 0])

'2. w 应放在最后：'
print tf.euler_from_quaternion([0.382683,0,0,0.92388,])
print tf.euler_from_quaternion([0.382683,0,0,0.92388,], axes='sxyz')

# 0.665976,0.482963,0.29995,0.482963
print tf.euler_from_quaternion([0.482963,0.29995,0.482963,0.665976,], axes='rxyz')

'0 四元数'
print tf.euler_from_quaternion([0, 0, 0, 1], axes='rxyz')
print tf.euler_from_quaternion([0, 0, 0, 1], axes='sxyz')


<type 'tuple'>
True
(0.7853970066908716, -0.0, 0.0)
(0.7853970066908716, -0.0, 0.0)
(0.7853981633974483, 1.0471968266705904, 0.7853981633974484)
(-0.0, 0.0, -0.0)
(0.0, -0.0, 0.0)


In [3]:
'''测试: 'ryzx', 是仅指定 yzx 旋转顺序呢， 还是同时指定 ai, aj, ak 分别代表的轴为 y, z, x 呢？ 后者 √
'''
eu = np.asarray([45,0,0])*np.pi/180
print np.array(tf.m2e(tf.e2m(*eu, axes='ryzx')[:3, :3], axes='rxyz'))*180/np.pi
print 'yzx:', tf.e2m(*eu, axes='ryzx')
print 'xyz:', tf.e2m(*eu, axes='rxyz')

'---------------another:'
eu = np.asarray([10,20,30])*np.pi/180
print '@@@:', tf.e2m(*eu, axes='rxyz')
eu2 = -np.asarray([30,20,10])*np.pi/180
print '@@@:', tf.e2m(*eu2, axes='rzyx').T #inversse
print np.allclose(tf.e2m(*eu, axes='rxyz'), tf.e2m(*eu2, axes='rzyx').T)
# tf.e2m(*eu, axes='rxyz')-tf.e2m(*eu, axes='rzyx').T

[  0.  45.  -0.]
yzx: [[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          1.         -0.          0.        ]
 [-0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]
xyz: [[ 1.         -0.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678  0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]
@@@: [[ 0.81379768 -0.46984631  0.34202014  0.        ]
 [ 0.54383814  0.82317294 -0.16317591  0.        ]
 [-0.20487413  0.31879578  0.92541658  0.        ]
 [ 0.          0.          0.          1.        ]]
@@@: [[ 0.81379768 -0.46984631  0.34202014  0.        ]
 [ 0.54383814  0.82317294 -0.16317591  0.        ]
 [-0.20487413  0.31879578  0.92541658  0.        ]
 [ 0.          0.          0.          1.        ]]
True


In [27]:
'测试： 欧拉角 -> 四元数， 一一对应， 还是多对一？ 万向锁'
e1 = np.asarray([30, 90, -40])*np.pi/180
e2 = np.asarray([70, 90, 0])*np.pi/180
print np.allclose(tf.e2q(*e1, axes='sxyz'), tf.e2q(*e2, axes='sxyz'))

e1 = np.asarray([30, 90, -40])*np.pi/180
e2 = np.asarray([-10, 90, 0])*np.pi/180
print np.allclose(tf.e2q(*e1, axes='rxyz'), tf.e2q(*e2, axes='rxyz'))
# print tf.e2q(*e2, axes='rxyz')

'e->q->e2， e!=e2， 一个四元数对应多个欧拉角'
e1 = np.asarray([30, 90, -40])*a2r
q1r = tf.e2q(*e1, axes='rxyz')
print np.asarray(tf.q2e(q1r, axes='rxyz'))*r2a

'e1->m->e2, e1!=e2, 一个旋转矩阵对应多个欧拉角'
m1 = tf.e2m(*e1, axes='rxyz')
print np.asarray(tf.m2e(m1, axes='rxyz'))*r2a

'确实， 一个旋转矩阵对应多个欧拉角'
# print tf.e2m(*e1, axes='rxyz')
# print tf.e2m(*e2, axes='rxyz')
# print tf.e2m(*e1, axes='rxyz') == tf.e2m(*e2, axes='rxyz')
print np.allclose(tf.e2m(*e1, axes='rxyz'), tf.e2m(*e2, axes='rxyz'))

"测试 q -> rm -> rm.T -> q' 与 q 关系: cos部不变， sin部取反， 注意 tf库是 xyzw 顺序， w在最后"
e1 = np.asarray([10,20,30])*a2r
q1 = tf.e2q(*e1, axes='rxyz')
m1 = tf.q2m(q1)
m2 = m1.T
q2 = tf.m2q(m2)
print q1, q2


True
True
[ -0.  90. -10.]
[ -0.  90. -10.]
True
[ 0.12767944  0.14487813  0.26853582  0.94371436] [-0.12767944 -0.14487813 -0.26853582  0.94371436]


In [44]:
e1 = np.asarray([10, 20, 30])*a2r
e2 = np.asarray([40, 20, 30])*a2r
q1 = tf.e2q(*e1, axes='rxyz')
q2 = tf.e2q(*e2, axes='rxyz')
print [q1[3]]+q1[:3].tolist() #w 放前面
tf.m2q(tf.q2m(q1))


[0.9437143641474891, 0.12767944069578063, 0.14487812541736916, 0.2685358227515692]


array([ 0.12767944,  0.14487813,  0.26853582,  0.94371436])

In [ ]:
m1 = tf.e2m

In [117]:
'测试： 两不同物体坐标系 a, b 在同一世界坐标系下，Rb -> Ra 怎么做？'

e1 = np.asarray([10,20,30])*np.pi/180
ma = tf.e2m(*e1, axes='rxyz')
# e2 = np.asarray([10,20,30])*np.pi/180
mb = tf.e2m(*e1, axes='ryzx')
print 'ma:\n', ma
print 'mb:\n', mb
mx = tf.e2m(np.pi/2, 0, 0, 'rxyz') #坐标系旋转 -90°， 物体旋转 +90°； tf.xxx 是旋转物体
mz = tf.e2m(0, 0, np.pi/2, 'rxyz')
print 'mx:', mx
print 'mz:', mz
Rab = np.dot(mz, mx)
print 'Rab:', Rab
# Rab = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,0,0,1]])
# print 'mx * mz:', Rab
mba = np.dot(Rab.T, mb).dot(Rab)
print 'mba:\n', mba
print 'mba-ma:', np.dot(ma.T, mba)
print np.array(tf.m2e(mb, 'ryzx'))*180/np.pi

ma:
[[ 0.81379768 -0.46984631  0.34202014  0.        ]
 [ 0.54383814  0.82317294 -0.16317591  0.        ]
 [-0.20487413  0.31879578  0.92541658  0.        ]
 [ 0.          0.          0.          1.        ]]
mb:
[[ 0.92541658 -0.20487413  0.31879578  0.        ]
 [ 0.34202014  0.81379768 -0.46984631  0.        ]
 [-0.16317591  0.54383814  0.82317294  0.        ]
 [ 0.          0.          0.          1.        ]]
mx: [[  1.00000000e+00   0.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   6.12323400e-17  -1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   1.00000000e+00   6.12323400e-17   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
mz: [[  6.12323400e-17  -1.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   6.12323400e-17   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
Rab:

In [7]:
'测试： cv2.FileStorage'
# cv2.cv.Load(extrinsics_xml) #cv1 只能用来 load iplimage
# 没文档， 不会写
fs = cv2.FileStorage(extrinsics_xml, cv2.FILE_STORAGE_READ)
rt = fs.root()
rt.size()

223L

In [210]:
'''
下面是另一些测试：
1. 测试一个旋转矩阵 list 统一左乘 R0.T 的结果，是否统一偏移了相同角度？
'''
cnt = 100
xang = (np.random.random(cnt) - .5) * np.pi * 2
yang = (np.random.random(cnt) - .5) * np.pi
zang = (np.random.random(cnt) - .5) * np.pi * 2
eus = np.asarray([xang, yang, zang])
# print eus
eus = eus.T
# print eus

In [212]:
'接上个 cell： 测试一个旋转矩阵 list 统一左乘 R0.T 的结果，是否统一偏移了相同角度？'
amode = 'sxyz'
# eus_mat = []
# for eu in eus:
#     eus_mat.append(tf.euler_matrix(*eu, axes=amode))
# eus_mat = np.asarray(eus_mat)
eus_mat = np.asarray([tf.euler_matrix(*eu, axes=amode) for eu in eus])
eus_mat2 = eus_mat.copy()

em0 = eus_mat[0]
for i, emi in enumerate(eus_mat):
#     emi = np.dot(em0.T, emi) #local, wrong ×
    eus_mat2[i] = np.dot(em0.T, emi)
    print '--', eus_mat2[i]
    pass

eus2 = np.asarray([tf.euler_from_matrix(eum, axes=amode) for eum in eus_mat2])

which_axis = 0
figure('eus')
plot(eus[:, which_axis])
figure('eus2')
plot(eus2[:, which_axis])


-- [[  1.00000000e+00   0.00000000e+00  -2.77555756e-17   0.00000000e+00]
 [  0.00000000e+00   1.00000000e+00   5.55111512e-17   0.00000000e+00]
 [ -2.77555756e-17   5.55111512e-17   1.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
-- [[-0.69513184  0.22299768 -0.68342063  0.        ]
 [ 0.14738731 -0.88626706 -0.4390987   0.        ]
 [-0.70361119 -0.40595902  0.58320542  0.        ]
 [ 0.          0.          0.          1.        ]]
-- [[ 0.75757559  0.60877803  0.23551758  0.        ]
 [-0.63999784  0.76370847  0.08457033  0.        ]
 [-0.12838221 -0.21479916  0.96818352  0.        ]
 [ 0.          0.          0.          1.        ]]
-- [[ 0.62390541  0.42772093 -0.65406181  0.        ]
 [ 0.75924309 -0.13342766  0.63698272  0.        ]
 [ 0.18518091 -0.89400888 -0.40799039  0.        ]
 [ 0.          0.          0.          1.        ]]
-- [[-0.90530255  0.36717758 -0.21356011  0.        ]
 [ 0.39919708  0.56364658 -0.72314

In [225]:
'(20,20,20).T * (60,60,60) 不等于 (40,40,40), 无论 sxyz, rxyz'
'欧拉角是不可传递的'

# e1 = np.asarray([np.pi/9]*3)
# e2 = np.asarray([np.pi/3]*3)
e1 = np.asarray([np.pi/9, np.pi/9, 0])
e21 = np.asarray([np.pi/3, np.pi/3, 0])
e22 = np.asarray([np.pi/3, np.pi/2, 0])

m1 = tf.e2m(*e1)
m21 = tf.e2m(*e21)
m31 = np.dot(m1.T, m21)
print np.array(tf.euler_from_matrix(m31))*180/np.pi
m22 = tf.euler_matrix(*e22)
m32 = np.dot(m1.T, m22)
print np.array(tf.euler_from_matrix(m32))*180/np.pi

m1 = tf.euler_matrix(*e1, axes='rxyz')
m21 = tf.euler_matrix(*e21, axes='rxyz')
m31 = np.dot(m1.T, m21)
print np.array(tf.euler_from_matrix(m31, axes='rxyz'))*180/np.pi
m22 = tf.euler_matrix(*e22, axes='rxyz')
m32 = np.dot(m1.T, m22)
print np.array(tf.euler_from_matrix(m32, axes='rxyz'))*180/np.pi


[ 34.58623312  37.15855414 -16.01289549]
[ 13.21917889  62.00910928 -43.21917889]
[ 26.09741558  43.06254932  17.51217232]
[  6.59354995e-15   7.00000000e+01   4.00000000e+01]


In [217]:
'global/local; static/rotating frame 的关系； from 杜宇笔记'
print tf.euler_matrix(10, 20, 30, axes='rxyz')
print tf.euler_matrix(30, 20, 10, axes='szyx')
print np.all(tf.euler_matrix(10, 20, 30, axes='rxyz')==tf.euler_matrix(30, 20, 10, axes='szyx'))


[[ 0.06294725  0.40319798  0.91294525  0.        ]
 [ 0.75241845 -0.62014526  0.22200526  0.        ]
 [ 0.65567074  0.67294223 -0.34241004  0.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.06294725  0.40319798  0.91294525  0.        ]
 [ 0.75241845 -0.62014526  0.22200526  0.        ]
 [ 0.65567074  0.67294223 -0.34241004  0.        ]
 [ 0.          0.          0.          1.        ]]
True
